In [9]:
import json
from http import HTTPStatus
import dashscope
from dotenv import dotenv_values
from retry import retry
config = dotenv_values('.env')

In [10]:
MODEL_NAME = 'qwen2-7b-instruct'
dashscope.api_key = config['qwen_key'],

@retry(delay=60, tries=3)
def call_qwen_api(MODEL_NAME, query):
    messages = [
        {'role': 'user', 'content': query}]
    response = dashscope.Generation.call(
        MODEL_NAME,
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    if response.status_code == HTTPStatus.OK:
        #print(response)
        return response['output']['choices'][0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        raise Exception()

In [11]:
def get_prompt(problem, question, options):

    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}

### 问题:
{question}
{options}
"""

    return prompt

In [24]:
def get_question_prompt(data):
    prompt = f"""# Role 
你是一个逻辑推理问题出题专家。以下是一个包含问题的 JSON 数据。请在原有子问题的基础上，额外生成三个新的子问题，保持格式一致并给出参考答案。

## Skills
1. 逻辑推理问题的设计与编写
2. 生成符合既定格式的题目
3. 提供高质量的参考答案

## Rules
1. 新生成的子问题需与原有问题逻辑一致。
2. 保持 JSON 数据的格式统一。
3. 提供的参考答案应清晰、准确。

## Workflows
1. 分析现有的 JSON 数据及其子问题。
2. 设计三个新的逻辑推理子问题。
3. 编写对应的参考答案。
4. 确保格式的一致性与准确性。

## Example
{data}
"""
    return prompt


In [12]:
def read_file(ifn):
    # 读取输入文件
    data = []
    with open(ifn) as reader:
        for line in reader:
            # 解析每一行JSON数据
            sample = json.loads(line)
            data.append(sample)

    return data

In [13]:
data = read_file('data/round1_train_data.jsonl')

In [26]:
query = get_question_prompt(data[0])
recall_qwen_api(MODEL_NAME, query)

'```json\n{\n  "problem": "假设有一个数学竞赛，参赛者需要解决一系列的数学问题。以下是一些关于分数和百分比的问题以及它们的正确答案。根据这些信息，设计以下问题并提供参考答案。",\n  "questions": [\n    {\n      "question": "选择题 1：\\n如果一个学生在一次考试中得了75分，而总分是100分，那么这个学生的分数是多少百分比？",\n      "options": ["A) 75%", "B) 100%", "C) 50%", "D) 37.5%"],\n      "answer": "A"\n    },\n    {\n      "question": "选择题 2：\\n如果一个数的50%等于另一个数的25%，那么这两个数的关系是什么？",\n      "options": ["A) 第一个数是第二个数的两倍", "B) 第一个数是第二个数的一半", "C) 两个数相等", "D) 无法确定"],\n      "answer": "A"\n    },\n    {\n      "question": "选择题 3：\\n一个数的20%减少了10个单位后，变成了30。原来的数是多少？",\n      "options": ["A) 50", "B) 40", "C) 60", "D) 70"],\n      "answer": "C"\n    }\n  ],\n  "id": "round1_train_data_001"\n}\n```\n\n在这个示例中，我们创建了三个新的逻辑推理子问题，涵盖了分数、百分比和数学关系的概念。这些问题设计得与原有的问题逻辑一致，遵循了JSON数据的格式，并提供了清晰、准确的参考答案。'